In [1]:
import pandas as pd
import numpy as np

In [42]:
path='/fs5/p_masi/rudravg/MxIF_Vxm_Registered_V2'
tissue_name='GCA022ACB_TISSUE02'
registered_intensities=pd.read_csv(f'{path}/{tissue_name}/registered_instances.csv')
unregistered_intensities=pd.read_csv(f'{path}/{tissue_name}/unregistered_instances.csv')
registered_thresholds=pd.read_csv(f'{path}/{tissue_name}/{tissue_name}_post_all_V2_registered.csv')
unregistered_thresholds=pd.read_csv(f'{path}/{tissue_name}/{tissue_name}_post_all_V2_unregistered.csv')

In [43]:
#Add a column called instances in registered thresholds and unregistered thresholds and start from 1 and go to end
registered_thresholds['instances'] = range(1, len(registered_thresholds) + 1)
unregistered_thresholds['instances'] = range(1, len(unregistered_thresholds) + 1)

In [44]:
registered_thresholds.head()

,nzNorm_Mean_CD11B,nzNorm_Mean_CD20,nzNorm_Mean_CD3d,nzNorm_Mean_CD45,nzNorm_Mean_CD4,nzNorm_Mean_CD68,nzNorm_Mean_CD8,nzNorm_Mean_CgA,nzNorm_Mean_Lysozyme,nzNorm_Mean_NaKATPase,nzNorm_Mean_PanCK,nzNorm_Mean_SMA,nzNorm_Mean_Sox9,nzNorm_Mean_Vimentin,nzNorm_Mean_OLFM4,instances
0,7.459612e-09,0.012197,0.072755,0.995196,0.002967,0.000015,0.000027,1.512048e-01,0.058607,3.841087e-11,8.815226e-09,0.519812,6.780402e-02,9.999943e-01,6.888423e-09,1
1,3.365999e-04,0.143256,0.818503,0.014184,0.004048,0.021744,0.014169,1.599057e-02,0.628168,3.479821e-05,5.428486e-02,0.999944,8.536146e-14,3.244000e-05,5.932985e-02,2
2,3.516540e-01,0.006640,0.933475,0.078522,0.000258,0.000660,0.020140,3.682841e-08,0.619122,1.707248e-06,1.559332e-02,0.999865,3.763030e-12,2.284380e-09,3.126853e-01,3
3,1.342137e-04,0.001583,0.001067,0.004341,0.007843,0.000096,0.012740,4.176389e-01,0.885051,3.675408e-06,7.736829e-02,1.000000,9.415093e-06,9.978876e-01,3.209160e-02,4
4,1.709306e-01,0.118057,0.921249,0.000000,0.038688,0.000034,0.026053,2.629972e-06,0.519318,1.526696e-07,7.740776e-01,0.999992,3.357705e-03,8.595918e-06,3.730402e-01,5


In [45]:
cols = ['instances']  + [col for col in registered_thresholds if col != 'instances']
registered_thresholds = registered_thresholds.reindex(columns=cols)

# Move the 'instances' column to the first position in unregistered_thresholds
cols = ['instances']  + [col for col in unregistered_thresholds if col != 'instances']
unregistered_thresholds = unregistered_thresholds.reindex(columns=cols)

In [46]:
registered_thresholds.head()

,instances,nzNorm_Mean_CD11B,nzNorm_Mean_CD20,nzNorm_Mean_CD3d,nzNorm_Mean_CD45,nzNorm_Mean_CD4,nzNorm_Mean_CD68,nzNorm_Mean_CD8,nzNorm_Mean_CgA,nzNorm_Mean_Lysozyme,nzNorm_Mean_NaKATPase,nzNorm_Mean_PanCK,nzNorm_Mean_SMA,nzNorm_Mean_Sox9,nzNorm_Mean_Vimentin,nzNorm_Mean_OLFM4
0,1,7.459612e-09,0.012197,0.072755,0.995196,0.002967,0.000015,0.000027,1.512048e-01,0.058607,3.841087e-11,8.815226e-09,0.519812,6.780402e-02,9.999943e-01,6.888423e-09
1,2,3.365999e-04,0.143256,0.818503,0.014184,0.004048,0.021744,0.014169,1.599057e-02,0.628168,3.479821e-05,5.428486e-02,0.999944,8.536146e-14,3.244000e-05,5.932985e-02
2,3,3.516540e-01,0.006640,0.933475,0.078522,0.000258,0.000660,0.020140,3.682841e-08,0.619122,1.707248e-06,1.559332e-02,0.999865,3.763030e-12,2.284380e-09,3.126853e-01
3,4,1.342137e-04,0.001583,0.001067,0.004341,0.007843,0.000096,0.012740,4.176389e-01,0.885051,3.675408e-06,7.736829e-02,1.000000,9.415093e-06,9.978876e-01,3.209160e-02
4,5,1.709306e-01,0.118057,0.921249,0.000000,0.038688,0.000034,0.026053,2.629972e-06,0.519318,1.526696e-07,7.740776e-01,0.999992,3.357705e-03,8.595918e-06,3.730402e-01


In [47]:
#For each column in the registered and unregistered thresholds starting from the second column, get the value closest to 0.5 or equal to 0.5. Should not be less than 0.5 and print the corresponding instance number for that row
for col in registered_thresholds.columns[1:]:
    registered_thresholds['instances'] = np.where((registered_thresholds[col] >= 0.5) & (registered_thresholds[col] < 0.6), registered_thresholds['instances'], registered_thresholds['instances'])
    unregistered_thresholds['instances'] = np.where((unregistered_thresholds[col] >= 0.5) & (unregistered_thresholds[col] < 0.6), unregistered_thresholds['instances'], unregistered_thresholds['instances'])



In [48]:
def closest_not_less(column):
    filtered = column[column >= 0.5]
    if not filtered.empty:
        return filtered.sub(0.5).abs().idxmin()
    return None

# Applying the function to each column, starting from the second column
results = {col: registered_thresholds.iloc[closest_not_less(registered_thresholds[col])] if closest_not_less(registered_thresholds[col]) is not None else None for col in registered_thresholds.columns[1:-1]}

# Print results
for col, row in results.items():
    if row is not None:
        print(f'Closest value to 0.5 or greater in {col}: {row[col]}, corresponding "instances" value: {row["instances"]}')
    else:
        print(f'No values are 0.5 or greater in {col}')


Closest value to 0.5 or greater in nzNorm_Mean_CD11B: 0.500195544390512, corresponding "instances" value: 560.0
Closest value to 0.5 or greater in nzNorm_Mean_CD20: 0.50002981087545, corresponding "instances" value: 285.0
Closest value to 0.5 or greater in nzNorm_Mean_CD3d: 0.500116626651641, corresponding "instances" value: 2273.0
Closest value to 0.5 or greater in nzNorm_Mean_CD45: 0.500365292299837, corresponding "instances" value: 11101.0
Closest value to 0.5 or greater in nzNorm_Mean_CD4: 0.500034715833438, corresponding "instances" value: 4212.0
Closest value to 0.5 or greater in nzNorm_Mean_CD68: 0.500341526009631, corresponding "instances" value: 11719.0
No values are 0.5 or greater in nzNorm_Mean_CD8
No values are 0.5 or greater in nzNorm_Mean_CgA
Closest value to 0.5 or greater in nzNorm_Mean_Lysozyme: 0.500012113036949, corresponding "instances" value: 11915.0
Closest value to 0.5 or greater in nzNorm_Mean_NaKATPase: 0.501593059682938, corresponding "instances" value: 5761.0

In [82]:
registered_intensities.loc[registered_intensities['Instance'] == 285]


,Instance,Centroid_X,Centroid_Y,Mean_CD11B,Mean_CD20,Mean_CD3d,Mean_CD45,Mean_CD4,Mean_CD68,Mean_CD8,Mean_CgA,Mean_Lysozyme,Mean_NaKATPase,Mean_PanCK,Mean_SMA,Mean_Sox9,Mean_Vimentin,Mean_OLFM4,slide_id
284,285,619.232779,743.052257,1.783848,6.660333,0.014252,0.249406,0.083135,0.0,0.432304,0.054632,6.503563,0.553444,79.952494,3.019002,0.002375,0.0,0.16152,registered


In [49]:
def closest_not_less(column):
    filtered = column[column >= 0.5]
    if not filtered.empty:
        return filtered.sub(0.5).abs().idxmin()
    return None

# Applying the function to each column, starting from the second column
results = {col: unregistered_thresholds.iloc[closest_not_less(unregistered_thresholds[col])] if closest_not_less(unregistered_thresholds[col]) is not None else None for col in unregistered_thresholds.columns[1:-1]}

# Print results
for col, row in results.items():
    if row is not None:
        print(f'Closest value to 0.5 or greater in {col}: {row[col]}, corresponding "instances" value: {row["instances"]}')
    else:
        print(f'No values are 0.5 or greater in {col}')

Closest value to 0.5 or greater in nzNorm_Mean_CD11B: 0.50004714098323, corresponding "instances" value: 3348.0
Closest value to 0.5 or greater in nzNorm_Mean_CD20: 0.500259441122933, corresponding "instances" value: 8671.0
Closest value to 0.5 or greater in nzNorm_Mean_CD3d: 0.500176697250684, corresponding "instances" value: 4631.0
Closest value to 0.5 or greater in nzNorm_Mean_CD45: 0.500279409686062, corresponding "instances" value: 3593.0
Closest value to 0.5 or greater in nzNorm_Mean_CD4: 0.50001222936509, corresponding "instances" value: 5313.0
Closest value to 0.5 or greater in nzNorm_Mean_CD68: 0.500486243433874, corresponding "instances" value: 1494.0
Closest value to 0.5 or greater in nzNorm_Mean_CD8: 0.501678095172459, corresponding "instances" value: 7456.0
No values are 0.5 or greater in nzNorm_Mean_CgA
Closest value to 0.5 or greater in nzNorm_Mean_Lysozyme: 0.500187886588796, corresponding "instances" value: 5616.0
Closest value to 0.5 or greater in nzNorm_Mean_NaKATPas

In [83]:
#Print the values in the row corresponding to th the Instances column which has 3348 value in registered intensities
unregistered_intensities.loc[unregistered_intensities['Instance'] == 8671]


,Instance,Centroid_X,Centroid_Y,Mean_CD11B,Mean_CD20,Mean_CD3d,Mean_CD45,Mean_CD4,Mean_CD68,Mean_CD8,Mean_CgA,Mean_Lysozyme,Mean_NaKATPase,Mean_PanCK,Mean_SMA,Mean_Sox9,Mean_Vimentin,Mean_OLFM4,slide_id
8670,8671,7144.86,4424.013333,1.02,2.655556,7.88,0.162222,1.315556,0.06,2.495556,0.288889,0.004444,1.157778,1.802222,0.0,20.8,0.0,1.117778,unregistered
